Introduction (What we want to do and what data we are looking at):
A novel corona virus (COVID 19) was identified in 2019 in Wuhan China. It has spread rapidly worldwide and was officially declared to be a pandemic by the WHO. To better understand the data available about it, we will be doing exploratory data analysis of the available COVID 19 data. The goal of the project is to study the impact of COVID 19 across the world using Python, Pandas and Matplotlib and present visualizations to show our analysis.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress

In [2]:
# File to Load 
file = "Data/owid-covid-data.csv"

# Read Purchasing File and store into Pandas data frame
data = pd.read_csv(file)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181463 entries, 0 to 181462
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    181463 non-null  object 
 1   continent                                   170857 non-null  object 
 2   location                                    181463 non-null  object 
 3   date                                        181463 non-null  object 
 4   total_cases                                 174693 non-null  float64
 5   new_cases                                   174468 non-null  float64
 6   new_cases_smoothed                          173299 non-null  float64
 7   total_deaths                                156469 non-null  float64
 8   new_deaths                                  156467 non-null  float64
 9   new_deaths_smoothed                         155320 non-null  float64
 

In [4]:
data.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [5]:
data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [6]:
to_drop = ['iso_code', 'new_cases_smoothed', 'new_deaths_smoothed', 'new_cases_smoothed_per_million', 
          'new_deaths_smoothed_per_million', 'reproduction_rate', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
          'new_people_vaccinated_smoothed','new_people_vaccinated_smoothed_per_hundred', 'stringency_index'
          ]

data.drop(to_drop, inplace=True, axis=1)

data.head()

,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,0.126,0.126,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,0.126,0.000,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,0.126,0.000,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,0.126,0.000,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,0.126,0.000,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [7]:
# number of countries
count_countries = data['location'].nunique()

# number of continents
count_continents = data['continent'].nunique()

print("This research includes data from ", count_countries, "countries from", count_continents, "continents.")

This research includes data from  243 countries from 6 continents.


In [8]:
# sample data, 50,000 rows of 181,463 rows
sample_data = data.sample(n=50000)
sample_data.head()

,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
142478,Asia,Saudi Arabia,2021-03-21,385020.0,367.0,6609.0,7.0,10894.527,10.385,187.008,...,1.8,25.4,NaN,2.7,75.13,0.854,NaN,NaN,NaN,NaN
80413,Europe,Isle of Man,2021-05-10,1590.0,0.0,29.0,0.0,18616.087,0.000,339.539,...,NaN,NaN,NaN,NaN,81.40,NaN,NaN,NaN,NaN,NaN
174601,Asia,Uzbekistan,2021-06-30,110677.0,487.0,736.0,3.0,3261.367,14.351,21.688,...,1.3,24.7,NaN,4.0,71.72,0.720,22281.4,9.56,18.6,656.57574
137735,North America,Saint Kitts and Nevis,2022-04-18,5559.0,0.0,43.0,0.0,103817.279,0.000,803.048,...,NaN,NaN,NaN,2.3,76.23,0.779,NaN,NaN,NaN,NaN
54234,NaN,European Union,2021-12-12,49685653.0,173459.0,867797.0,919.0,111106.381,387.887,1940.556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Research questions to answer:


Correlation between cases and amount of people vaccinated

GDP versus vaccines/cases 

Relation between rate of vaccination and mortality rate

Relation between rate of vaccination and mortality rate 

Age nd mortality rate 

Age and vaccination rate 

How effective the vaccination is in each country

New cases over time by country

Hospitalization rate by country

If people with preexisting conditions are more prone to be hospitalized or die